<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Speeach to text con NVIDIA NeMo
Ejemplo basado en [LINK](https://github.com/NVIDIA/NeMo)

### NVIDIA NeMo
Es un modelo entrenado para reconocimiento de voz, el cual puede:
- Transformar audio a texto
- Agregar puntiación o mayúsculas a texto

Modelos [LINK](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/core/core.html#)

Tutoriales [LINK](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/starthere/tutorials.html):
* [NeMo fundamentals](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/00_NeMo_Primer.ipynb)
* [NeMo models](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/01_NeMo_Models.ipynb)
* [Speech Recognition](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/asr/01_ASR_with_NeMo.ipynb)
* [Punctuation and Capitalization](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/nlp/Punctuation_and_Capitalization.ipynb)
* [Speech Synthesis](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/tts/1_TTS_inference.ipynb)

## Installation
Installation of NeMo and its dependencies.

In [ ]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

In [ ]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing colleciton
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# We'll use this to listen to audio
import IPython

[NeMo W 2021-09-24 00:32:34 optimizers:47] Apex was not found. Using the lamb optimizer will error out.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


[NeMo W 2021-09-24 00:32:39 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-09-24 00:32:43 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.decoder_dataset.TextNormalizationDecoderDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-09-24 00:32:43 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.tagger_dataset.TextNormalizationTaggerDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-09-24 00:32:43 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.test_dataset.TextNormalizationTestDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[NeMo W 2021-09-24 00:32:44 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_decoder.DuplexDecoderModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-09-24 00:32:44 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_tagger.DuplexTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[NeMo W 2021-09-24 00:32:44 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_tn.DuplexTextNormalizationModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [ ]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2021-09-24 00:32:50--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.99.42
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.99.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K  --.-KB/s    in 0.1s    

2021-09-24 00:32:51 (2.01 MB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



## Instantiate pre-trained NeMo models which we'll use
``from_pretrained(...)`` API downloads and initialized model directly from the cloud.

We will load audio_sample and convert it to text with QuartzNet ASR model (an action called transcribe).
To convert text back to audio, we actually need to generate spectrogram with Tacotron2 first and then convert it to actual audio signal using WaveGlow vocoder.

In [ ]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="tts_en_tacotron2").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda()

[NeMo I 2021-09-24 00:33:17 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.2.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2021-09-24 00:33:20 common:676] Instantiating model from pre-trained checkpoint


[NeMo W 2021-09-24 00:33:21 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2021-09-24 00:33:21 modelPT:146] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2021-09-24 00:33:21 features:252] PADDING: 16
[NeMo I 2021-09-24 00:33:21 features:269] STFT using torch
[NeMo I 2021-09-24 00:33:32 modelPT:438] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.2.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2021-09-24 00:33:32 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/punctuation_en_distilbert/versions/1.0.0rc1/files/punctuation_en_distilbert.nemo to /root/.cache/torch/NeMo/NeMo_1.2.0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2021-09-24 00:33:40 common:676] Instantiating model from pre-trained checkpoint
[NeMo I 2021-09-24 00:33:45 tokenizer_utils:101] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmpgknoo7kk/tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

[NeMo I 2021-09-24 00:34:06 modelPT:438] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.2.0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.
[NeMo I 2021-09-24 00:34:06 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_tacotron2/versions/1.0.0/files/tts_en_tacotron2.nemo to /root/.cache/torch/NeMo/NeMo_1.2.0/tts_en_tacotron2/74d819609f47744063679b44e93a47f1/tts_en_tacotron2.nemo
[NeMo I 2021-09-24 00:34:10 common:676] Instantiating model from pre-trained checkpoint


[NeMo W 2021-09-24 00:34:12 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_train.json
      max_duration: null
      min_duration: 0.1
      trim: false
      int_values: false
      load_audio: true
      normalize: true
      sample_rate: 22050
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 48
      num_workers: 4
    
[NeMo W 2021-09-24 00:34:12 modelPT:146] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_

[NeMo I 2021-09-24 00:34:12 features:252] PADDING: 16
[NeMo I 2021-09-24 00:34:12 features:269] STFT using torch
[NeMo I 2021-09-24 00:34:12 modelPT:438] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.2.0/tts_en_tacotron2/74d819609f47744063679b44e93a47f1/tts_en_tacotron2.nemo.
[NeMo I 2021-09-24 00:34:12 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_waveglow_88m/versions/1.0.0/files/tts_waveglow.nemo to /root/.cache/torch/NeMo/NeMo_1.2.0/tts_waveglow/be8c69c04450b3da0ce5a38a8ef4014f/tts_waveglow.nemo
[NeMo I 2021-09-24 00:34:23 common:676] Instantiating model from pre-trained checkpoint


[NeMo W 2021-09-24 00:34:27 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_train.json
      max_duration: null
      min_duration: 0.1
      n_segments: 16000
      trim: false
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 12
      num_workers: 4
    
[NeMo W 2021-09-24 00:34:27 modelPT:146] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_val.json

[NeMo I 2021-09-24 00:34:27 features:252] PADDING: 16
[NeMo I 2021-09-24 00:34:27 features:269] STFT using torch
[NeMo I 2021-09-24 00:34:29 modelPT:438] Model WaveGlowModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.2.0/tts_waveglow/be8c69c04450b3da0ce5a38a8ef4014f/tts_waveglow.nemo.


## Using the models

In [ ]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f"\nRaw recognized text:\n {raw_text}")
print(f"\nText with capitalization and punctuation:\n {text}")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-09-24 00:34:30 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.


[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_model:384] Using batch size 1 for inference
[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_dataset:134] Max length: 27
[NeMo I 2021-09-24 00:34:30 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2021-09-24 00:34:30 data_preprocessing:364] Min: 27 |                  Max: 27 |                  Mean: 27.0 |                  Median: 27.0
[NeMo I 2021-09-24 00:34:30 data_preprocessing:366] 75 percentile: 27.00
[NeMo I 2021-09-24 00:34:30 data_preprocessing:367] 99 percentile: 27.00
[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_dataset:165] 0 are longer than 27
[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-09-24 00:34:30 punctuation_capitalization_dataset:170] subtokens: [CLS] well i don ' t wish to see it any more observed phoebe turning away her eyes it is certainly ver

In [ ]:
# A helper function which combines Tacotron2 and WaveGlow to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

## Results

In [ ]:
# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [ ]:
# This is what was recognized by the ASR model
print(raw_text)

In [ ]:
# This is how punctuation model changed it
print(text)

Compare how the synthesized audio sounds when using text with and without punctuation.

In [ ]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

In [ ]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)